In [1]:
%load_ext autoreload
%autoreload 2

import cv2, os
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split

images_path = os.path.join(os.getcwd(), './ImageSets')
annotations_path = os.path.join(os.getcwd(), './SegmentationClass')

images_name = os.listdir(images_path)
annotations_name = os.listdir(annotations_path)

images_name_train, images_name_test, annotations_name_train, annotations_name_test = train_test_split(images_name, annotations_name, train_size=0.8, random_state=42)

['938_IMG_20180911_214011.jpg', '1020_IMG_20180928_225945.jpg', '634_IMG_20180116_204337.jpg', '960_IMG_20180928_203050.jpg', '928_IMG_20180911_213444.jpg', 'bayan_box6_c52db94d-07bf-4c93-8b95-f5e11fae27dd.jpg', '914_IMG_20180911_212553.jpg', '937_IMG_20180911_213945.jpg', '939_IMG_20180911_214553.jpg', '933_IMG_20180911_213748.jpg']


In [6]:
from ThresholdModel import ThresholdModel

model = ThresholdModel(images_path, annotations_path)
model.add_label('DiaChua', [255, 221, 51])
model.add_label('VungVoKhuan', [102, 255, 102])
model.add_label('GiayKhangSinh', [240, 120, 240])

model.fit(images_name_train, annotations_name_train)

DiaChua [22.32865176 26.5294368  28.75540845] [ 94.12402998 105.11477611 111.13636997]
VungVoKhuan [24.44361592 31.05768595 34.75743175] [ 94.2651039  105.48941881 111.11809824]
GiayKhangSinh [23.12287935 30.29840479 33.33883135] [ 99.48786759 110.57353913 116.28823017]


In [7]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ThresholdModel import iou_measure

accuracy = {
    'name': [],
    'DiaChua': [],
    'VungVoKhuan': [],
    'GiayKhangSinh': []
}

for image_name, annotation_name in zip(images_name_test, annotations_name_test):
    image_test = cv2.imread(os.path.join(images_path, image_name))
    annotation_test = cv2.imread(os.path.join(annotations_path, annotation_name))
    
    mask_DiaChua = model.predict(image_test, 'DiaChua')
    mask_VungVoKhuan = model.predict(image_test, 'VungVoKhuan')
    mask_GiayKhangSinh = model.predict(image_test, 'GiayKhangSinh')
    
    accuracy['name'].append(image_name)
    accuracy['DiaChua'].append(iou_measure(mask_DiaChua, model.get_mask(annotation_test, 'DiaChua')))
    accuracy['VungVoKhuan'].append(iou_measure(mask_VungVoKhuan, model.get_mask(annotation_test, 'VungVoKhuan')))
    accuracy['GiayKhangSinh'].append(iou_measure(mask_GiayKhangSinh, model.get_mask(annotation_test, 'GiayKhangSinh')))

    print('%s %.4f %.4f %.4f' % (accuracy['name'][-1], accuracy['DiaChua'][-1], accuracy['VungVoKhuan'][-1], accuracy['GiayKhangSinh'][-1]))

output = pd.DataFrame(accuracy)
output.to_csv('output.csv')

938_IMG_20180911_214011.jpg 0.2456 0.5867 0.0349
1020_IMG_20180928_225945.jpg 0.2439 0.0405 0.0087


ValueError: operands could not be broadcast together with shapes (3968,2976,3) (4128,3096,3) 

In [25]:
output

,name,DiaChua,VungVoKhuan,GiayKhangSinh
0,879_IMG_20180911_195128.jpg,0.342546,0.205640,0.004145
1,873_IMG_20180911_194441.jpg,0.443808,0.647355,0.111040
2,870_IMG_20180911_194225.jpg,0.591059,0.517454,0.102696
3,880_IMG_20180911_195205.jpg,0.341821,0.413190,0.034984
4,868_IMG_20180911_194036.jpg,0.248475,0.788308,0.058259
5,nadia_box5_75c28c60-77e5-4236-9a1a-828f506caf1...,0.712071,0.205207,0.046006
6,877_IMG_20180911_194847.jpg,0.349340,0.447334,0.020887
7,hadeel_box6_3620256a-d309-44ad-9dd7-f649ecf9a8...,0.133761,0.347276,0.009725
8,882_IMG_20180911_195412.jpg,0.499081,0.654529,0.085512
